In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle
import numpy as np
import mdtraj as md
from timewarp.utils.evaluation_utils import compute_internal_coordinates
from timewarp.datasets import RawMolDynDataset
from timewarp.dataloader import (
    DenseMolDynBatch,
    moldyn_dense_collate_fn,
)
from timewarp.utils.tica_utils import tica_features, run_tica, plot_tic01, plot_free_energy
from itertools import islice
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm

plt.rc('font', size=35) 


In [ ]:
base_dir = "../../"
#data_dir = base_dir + f".data/simulated-data/trajectory-data/4AA-huge/val" 
data_dir = base_dir + f".data/simulated-data/trajectory-data/4AA-huge/test" 

protein = "LAKS"
#protein = "SAGA"
#protein = "GTSG"
#protein = "DSGA"
#protein = "DPAS"
#protein = "GNGA"
#protein = "RKGR"
#protein ="MGRS"
protein = "LPEM"

raw_dataset = RawMolDynDataset(data_dir=data_dir, step_width=10000)
pdb_names = [protein]
raw_iterator = raw_dataset.make_iterator(pdb_names)
batches = (moldyn_dense_collate_fn([datapoint]) for datapoint in raw_iterator)
batches = list(islice(batches, 200))  # Load max 20 timepoints.
batch = batches[0]
state0pdbpath = os.path.join(data_dir, f"{protein}-traj-state0.pdb")
parameters =  "T1B-peptides"
adj_list = batch.adj_list


In [ ]:
npz =  np.load(data_dir+f"/{protein}-traj-arrays.npz")
xyz =  npz['positions'][::5]

topology = md.load(state0pdbpath).topology

bonds, torsions = compute_internal_coordinates(
    state0pdbpath, adj_list.cpu().numpy(), xyz
)
trajectory = md.Trajectory(
    xyz=xyz,
    topology=topology
)

In [ ]:
# TICA can be loaded if constructed before

DO_LOAD = False

features = tica_features(trajectory)

if DO_LOAD:
    with open("tica_model.pic", "rb") as f:
        tica_model = pickle.load(f)
else:
    tica_model = run_tica(trajectory, lagtime=100)
    with open("tica_model.pic", "wb") as f:
        pickle.dump(tica_model, f)
        
tics = tica_model.transform(features)

## Analyze TICS

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plot_tic01(ax, tics, "MD", tics_lims=tics)

In [ ]:
for i in range(2):
    plt.figure(figsize=(16, 9))
    plt.plot(tics[:, i])
    plt.title(f"TIC {i} - MD")

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plot_free_energy(ax, tics, "tic0")

## Load Data for model

In [ ]:
def load_trajectory_from_dir(directory_name, protein, stride=1):
    directory = os.fsencode(directory_name)
    trajectories = []
    times = []
    for i in range(len(os.listdir(directory))-1):
        npz = np.load(directory_name+f"/{protein}_trajectory_model_{i}.npz")
        trajectories.append(npz['positions'][::stride])
        try:
            times.append(npz['time'])
        except:
            pass
    return np.concatenate(trajectories, axis=0), np.array(times)

In [ ]:
initial_state=1


sampled_coords, times = load_trajectory_from_dir(dir_model_samples, protein)
traj_model = md.Trajectory(
    xyz=sampled_coords,
    topology=topology
)


In [ ]:
_, idxs, counts = np.unique(sampled_coords[:, 0, 0], return_index=True, return_counts=True)
stuck_idx = np.where(counts > 50000)
idxs[stuck_idx]


In [ ]:
len(traj_model.xyz)

In [ ]:
acceptance = (len(np.unique(traj_model.xyz[:, 0, 0]))-1)/len(traj_model.xyz) / 10
acceptance 

In [ ]:
feat_model = tica_features(traj_model)
tics_model = tica_model.transform(feat_model)
fig, ax = plt.subplots(figsize=(10,10))
plot_tic01(ax, tics_model, "model", tics_lims=tics)


In [ ]:
for i in range(2):
    plt.figure(figsize=(16, 9))
    plt.plot(tics_model[:, i])
    plt.title(f"TIC {i} - MD")

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plot_tic01(ax, tics, "MD", tics_lims=tics)
plot_tic01(ax, tics_model, f"{protein} MD - model", tics_lims=tics, cmap='autumn')


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plot_free_energy(ax, tics, "tic0", "MD")
plot_free_energy(ax, tics_model, "tic0", "model")
ax.legend();
fig, ax = plt.subplots(figsize=(10,10))
plot_free_energy(ax, tics, "tic1", "MD", axis=1)
plot_free_energy(ax, tics_model, "tic1", "model", axis=1)
ax.legend();

## Ramachandran plot

In [ ]:
def plot_ramachandrans(torsions, name):
    fig, axes = plt.subplots(1, 3, figsize=(33, 10), sharey=True )
    for i in range(3):
        #plt.title(f"Ramachandran plot - model")
        axes[i].hist2d(torsions[0][:, i], torsions[1][:, i], bins=100, norm=mpl.colors.LogNorm())
        axes[i].scatter(torsions[0][0, i], torsions[1][0, i], marker="x", color="red", s=500, linewidths=5)
        axes[i].set_xlim(-np.pi,np.pi)
        axes[i].set_ylim(-np.pi,np.pi)
        axes[i].set_xlabel("Phi")
        axes[0].set_ylabel("Psi")
    fig.suptitle(f"Ramachandran plots - {name}")


In [ ]:
bonds_model_sampled, torsions_model_sampled = compute_internal_coordinates(
    state0pdbpath, adj_list.cpu().numpy(), traj_model.xyz
)
plot_ramachandrans(torsions_model_sampled, "model")
plot_ramachandrans(torsions, "MD")

# Speed-up

In [ ]:
import arviz as az

def ESS(autocorrelations, spacing=1, cut_off_at_zero=True):
    if cut_off_at_zero:
        steps_until_zero = np.where(autocorrelations<=0)[0][0]
    else:
        steps_until_zero = -1
    Neff = 1/(-1 + 2 * spacing * np.abs(autocorrelations[:steps_until_zero]).sum())
    return Neff

In [ ]:
model_time_per_step = times.sum() / (len(sampled_coords) *10)
# Check if it is a new MD run, where the time gets saved
try:
    time_md = np.load(data_dir + f"/{protein}-traj-time.npy")
    md_time_per_step = time_md / (2*10**9)
except FileNotFoundError:
    print("Using avarage time for MD simulation")
    md_time_per_step = 0.00011304
    
model_time_per_step, md_time_per_step

In [ ]:
# nuber of MD steps due to spacing (10000 for all 2AA and 4AA)
num_md_steps = 10000

# number of staeps to plot
max_md_steps= 10000
num_model_steps = 100000

# TIC 1
autocorrelation_model = az.autocorr(tics_model[:, 1])
autocorrelation_openMM = az.autocorr(tics[:, 1])
ess_model_s = ESS(autocorrelation_model, spacing=1)/(model_time_per_step * 10)
ess_md_s = ESS(autocorrelation_openMM, spacing=1)/(md_time_per_step * num_md_steps)
plt.figure(figsize=(16,9))
plt.plot(np.arange(0, num_model_steps) * model_time_per_step * 10, autocorrelation_model[:num_model_steps], label=f"model | ESS/t = {ess_model_s:.3f}/s", linewidth=5)
plt.plot(np.arange(0, max_md_steps) * md_time_per_step * num_md_steps, autocorrelation_openMM[:max_md_steps], label=f"MD | ESS/t = {ess_md_s:.3f}/s", linewidth=5)
plt.axhline(0, color='black')
plt.semilogx()
plt.ylabel("TIC 1 autocorrelation")
plt.title("TIC 1 autocorrelations")
plt.xlabel("Lag time in s")
plt.legend();


# TIC 0
autocorrelation_model = az.autocorr(tics_model[:, 0])
autocorrelation_openMM = az.autocorr(tics[:, 0])
ess_model_s = ESS(autocorrelation_model, spacing=1)/(model_time_per_step * 10)
ess_md_s = ESS(autocorrelation_openMM, spacing=1)/(md_time_per_step * num_md_steps)
plt.figure(figsize=(16,9))
plt.plot(np.arange(0, num_model_steps) * model_time_per_step * 10, autocorrelation_model[:num_model_steps], label=f"model | ESS/t = {ess_model_s:.3f}/s", linewidth=5)
plt.plot(np.arange(0, max_md_steps) * md_time_per_step * num_md_steps, autocorrelation_openMM[:max_md_steps], label=f"MD | ESS/t = {ess_md_s:.3f}/s", linewidth=5)
plt.axhline(0, color='black')
plt.semilogx()
plt.ylabel("TIC 0 autocorrelation")
plt.title("TIC 0 autocorrelations")
plt.xlabel("Lag time in s")
plt.legend();

plt.figure(figsize=(16,9))
plt.plot(np.arange(0, num_model_steps) * model_time_per_step * 10, autocorrelation_model[:num_model_steps], label=f"model | ESS/t = {ess_model_s:.3f}/s", linewidth=5)
plt.plot(np.arange(0, max_md_steps) * md_time_per_step * num_md_steps, autocorrelation_openMM[:max_md_steps], label=f"MD | ESS/t = {ess_md_s:.3f}/s", linewidth=5)
plt.axhline(0, color='black')
plt.ylabel("TIC 0 autocorrelation")
plt.title("TIC 0 autocorrelations")
plt.xlabel("Lag time in s")
plt.legend();

In [ ]:
assert False

## Save for paper figures

In [ ]:
save = False


In [ ]:
from matplotlib.colors import LogNorm
plt.rc('font', size=35) 

def plot_tic01_2(ax, tics, tics_lims=tics, cmap='viridis'):
    _ = ax.hist2d(tics[:,0], tics[:,1], bins=100, norm=LogNorm(), cmap=cmap, rasterized=True)

    ax.set_ylim(tics_lims[:,1].min(),tics_lims[:,1].max())
    ax.set_xlim(tics_lims[:,0].min(),tics_lims[:,0].max())

    
def plot_free_energy2(ax, torison, label=None, linestyle='-'):
    hist, edges = np.histogram(torison, bins=100, density=True)
    free_energy = -np.log(hist/hist.max())
    centers = 0.5*(edges[1:] + edges[:-1])
    ax.plot(centers, free_energy, linewidth=5, label=label, linestyle=linestyle)
    #ax.set_xlabel(xlabel)
    #plt.title("Free energy")
    

In [ ]:

tics_model_exp = exploration['tics_model']
energies_model_exp = exploration['energies_model']


In [ ]:
from matplotlib.ticker import FormatStrFormatter

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True, sharex=True)
plot_tic01_2(axes[0], tics)
plot_tic01_2(axes[1], tics_model_exp)
plot_tic01_2(axes[2], tics_model)
axes[0].set_xlabel("TIC 0")
axes[1].set_xlabel("TIC 0")
axes[2].set_xlabel("TIC 0")
axes[0].set_ylabel("TIC 1")
axes[0].set_title("MD")
axes[1].set_title("Exploration")
axes[2].set_title("MCMC")
axes[0].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.subplots_adjust(wspace=0, hspace=0)



In [ ]:
MCMC = True
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
plot_free_energy2(axes[0], tics[:, 0],"MD")
plot_free_energy2(axes[0], tics_model_exp[:, 0],"Exploration", linestyle="-.")
if MCMC:
    plot_free_energy2(axes[0], tics_model[:, 0],"MCMC", linestyle="--")
plot_free_energy2(axes[1], tics[:, 1],"MD")
plot_free_energy2(axes[1], tics_model_exp[:, 1],"Exploration", linestyle="-.")
if MCMC:
    plot_free_energy2(axes[1], tics_model[:, 1],"MCMC", linestyle="--")
#axes[0].set_xlim(-np.pi,np.pi)
axes[0].set_xlabel("TIC 0")
axes[0].set_ylabel("Free energy")
#axes[0].set_ylabel("F/k_B T")
axes[1].set_xlabel("TIC 1")
axes[1].xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
axes[1].xaxis.set_major_locator(plt.MaxNLocator(2))

axes[1].legend(fontsize=25)


In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(11, 6), sharey=True, sharex=True)
sampling_time_md = np.arange(len(tics)) * md_time_per_step * num_md_steps / 3600
sampling_time_model = np.arange(len(tics_model)) * model_time_per_step * 10 /3600
max_time = min(sampling_time_md[-1],sampling_time_model[-1])
axs[0].plot(sampling_time_md[sampling_time_md<max_time], tics[sampling_time_md<max_time, 0], linewidth=.1)
axs[1].plot(sampling_time_model[sampling_time_model<max_time], tics_model[sampling_time_model<max_time, 0], linewidth=.1)
axs[1].set_xlabel("Wall-clock sampling time in h")
axs[1].set_ylabel("TIC 0")
axs[0].set_ylabel("TIC 0")
axs[0].set_title("MD")
axs[1].set_title("model")


In [ ]:

plt.figure(figsize=(13,6))
plt.plot(np.arange(0, max_md_steps) * md_time_per_step * num_md_steps, autocorrelation_openMM[:max_md_steps], label=f"MD | ESS/t = {ess_md_s:.4f}/s", linewidth=5)
plt.plot(np.arange(0, num_model_steps) * model_time_per_step * 10, autocorrelation_model[:num_model_steps], label=f"model | ESS/t = {ess_model_s:.4f}/s", linewidth=5)
plt.axhline(0, color='black')
plt.semilogx()
plt.ylabel("TIC 0 autocorrelation")
plt.title("TIC 0 autocorrelations")
plt.xlabel("Lag time in s")
plt.legend();


In [ ]:
# energies
from simulation.md import (
    get_simulation_environment,
    compute_energy_and_forces,
    compute_energy_and_forces_decomposition,
    get_parameters_from_preset, 
    get_simulation_environment_integrator,
    get_simulation_environment_for_force
)
import torch
from timewarp.utils.openmm import OpenmmPotentialEnergyTorch

simulation = get_simulation_environment(state0pdbpath, parameters)
integrator = get_simulation_environment_integrator(parameters)
system = simulation.system

openmm_potential_energy_torch = OpenmmPotentialEnergyTorch(system, integrator, platform_name='CUDA')



energies_model = openmm_potential_energy_torch(torch.from_numpy(traj_model.xyz[::10]))
traj_energy = npz['energies'][::5, 0]

plt.figure(figsize=(12, 6))
plt.hist(
    traj_energy,
    bins=100,
    density=True,
    label="MD",
    histtype='step',
    linewidth=5,
    #range=lims,
    #alpha=0.5,
)

plt.hist(
    energies_model_exp,
    bins=100,
    density=True,
    label=f"Exploration",
    histtype='step',
    #range=lims,
    #linestyle='-.',
    linewidth=5,
    #alpha=0.5,
)
plt.hist(
    energies_model.cpu().numpy().flatten(),
    bins=100,
    density=True,
    label=f"MCMC",
    histtype='step',
    linestyle='-',
    #range=lims,
    linewidth=5,
    #alpha=0.5,
)
plt.title("Energy distribution")
plt.xlabel("Energy in kJ/mol")
plt.legend();


In [ ]:
from scipy.spatial import distance


distance.cdist(tics[::10, :2],tics_model[::10, :2]).min(axis=1).max()

In [ ]:
ranges = np.abs(tics[::10, :2]).max(0)
distance.cdist(tics[::10, :2]/ranges,tics_model[::10, :2]/ranges).min(axis=1).max()

In [ ]:
assert False

## Presentation figures

In [ ]:
model_exp_time_per_step = time/10000

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12, 9), sharey=True, sharex=True)
sampling_time_md = np.arange(len(tics)) * md_time_per_step * num_md_steps / 3600
sampling_time_model = np.arange(len(tics_model)) * model_time_per_step * 10 /3600
sampling_time_model_exp = np.arange(len(tics_model_exp)) * model_exp_time_per_step/3600

max_time = min(sampling_time_md[-1],sampling_time_model[-1], 1.5)
axs[0].plot(sampling_time_md[sampling_time_md<max_time], tics[sampling_time_md<max_time, 0], linewidth=.1)
axs[1].plot(sampling_time_model[sampling_time_model<max_time], tics_model[sampling_time_model<max_time, 0], linewidth=.1)
axs[2].plot(sampling_time_model_exp[sampling_time_model_exp<max_time], tics_model_exp[sampling_time_model_exp<max_time, 0], linewidth=.1)

axs[2].set_xlabel("Wall-clock sampling time in h")
axs[1].set_ylabel("TIC 0")
axs[0].set_ylabel("TIC 0")
axs[0].set_title("MD")
axs[1].set_title("model")


In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12, 9), sharey=True, sharex=True)
sampling_time_md = np.arange(len(tics)) * md_time_per_step * num_md_steps / 3600
sampling_time_model = np.arange(len(tics_model)) * model_time_per_step * 10 /3600
sampling_time_model_exp = np.arange(len(tics_model_exp)) * model_exp_time_per_step/3600

max_time = min(sampling_time_md[-1],sampling_time_model[-1], .5)
axs[0].plot(sampling_time_md[sampling_time_md<max_time], tics[sampling_time_md<max_time, 1], linewidth=.1)
axs[1].plot(sampling_time_model[sampling_time_model<max_time], tics_model[sampling_time_model<max_time, 1], linewidth=.1)
axs[2].plot(sampling_time_model_exp[sampling_time_model_exp<max_time], tics_model_exp[sampling_time_model_exp<max_time, 1], linewidth=.1)

axs[2].set_xlabel("Wall-clock sampling time in h")
axs[1].set_ylabel("TIC 0")
axs[0].set_ylabel("TIC 0")
axs[0].set_title("MD")
axs[1].set_title("model")


In [ ]:
%matplotlib notebook

In [ ]:
def plot_tic01_background(ax, tics, tics_lims=tics, cmap='viridis'):
    _ = ax.hist2d(tics[:,0], tics[:,1], bins=100, norm=LogNorm(), cmap=cmap, rasterized=True)

    ax.set_ylim(tics_lims[:,1].min(),tics_lims[:,1].max())
    ax.set_xlim(tics_lims[:,0].min(),tics_lims[:,0].max())


In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.set_ylim(tics[:,1].min(),tics[:,1].max())
ax.set_xlim(tics[:,0].min(),tics[:,0].max())
ax.set_xticks([])
ax.set_yticks([])
plot_tic01_background(ax, tics_model_exp, tics)
plt.savefig(base_dir+f"outputs/figures/4AA-model-backgorund-{protein}.svg", bbox_inches = "tight")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.set_ylim(tics[:,1].min(),tics[:,1].max())
ax.set_xlim(tics[:,0].min(),tics[:,0].max())
ax.set_xticks([])
ax.set_yticks([])
plot_tic01_background(ax, tics_model_exp, tics)
plt.savefig(base_dir+f"outputs/figures/4AA-model-backgorund-{protein}.svg", bbox_inches = "tight")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.set_ylim(tics[:,1].min(),tics[:,1].max())
ax.set_xlim(tics[:,0].min(),tics[:,0].max())
ax.set_xticks([])
ax.set_yticks([])
plot_tic01_background(ax, tics_model, tics)
plt.savefig(base_dir+f"outputs/figures/4AA-model-backgorund-{protein}-model.svg", bbox_inches = "tight")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.set_ylim(tics[:,1].min(),tics[:,1].max())
ax.set_xlim(tics[:,0].min(),tics[:,0].max())
ax.set_xticks([])
ax.set_yticks([])
plot_tic01_background(ax, tics, tics)
plt.savefig(base_dir+f"outputs/figures/4AA-md-backgorund-{protein}.svg", bbox_inches = "tight")

In [ ]:
#num_tics_model = len(tics_model[sampling_time_model<max_time])
md_sapcing = 10
tics_model_short = tics_model_exp[sampling_time_model_exp<max_time][2::10][:-53]
tics_short = tics[sampling_time_md<max_time][::md_sapcing]
num_tics_short = len(tics_short)

In [ ]:
#num_tics_model = len(tics_model[sampling_time_model<max_time])
md_sapcing = 10
tics_model_short = tics_model[sampling_time_model<max_time][2::19][:-30]


In [ ]:
len(tics_model_short), num_tics_short

In [ ]:
tics_model_short[:, 2].var(0)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.set_ylim(tics[:,1].min(),tics[:,1].max())
ax.set_xlim(tics[:,0].min(),tics[:,0].max())
ax.set_xticks([])
ax.set_yticks([])
ax.scatter(tics_model_short[:, 0], tics_model_short[:, 1], c="C1", alpha=0.85)
ax.plot(tics_model_short[:, 0], tics_model_short[:, 1], c="C1", alpha=0.85)
plt.savefig(base_dir+f"outputs/figures/Transitions-final-{protein}-model-exp.svg", bbox_inches = "tight")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.set_ylim(tics[:,1].min(),tics[:,1].max())
ax.set_xlim(tics[:,0].min(),tics[:,0].max())
ax.set_xticks([])
ax.set_yticks([])
ax.scatter(tics_short[:, 0], tics_short[:, 1], c="C0", alpha=0.85)
ax.plot(tics_short[:, 0], tics_short[:, 1], c="C0", alpha=0.85)
plt.savefig(base_dir+f"outputs/figures/Transitions-final-{protein}-md.svg", bbox_inches = "tight")

In [ ]:

import matplotlib.animation as animation
import numpy as np
import itertools 


fig, ax = plt.subplots(figsize=(4,4))
ax.set_ylim(tics[:,1].min(),tics[:,1].max())
ax.set_xlim(tics[:,0].min(),tics[:,0].max())
ax.set_xticks([])
ax.set_yticks([])
#ax.set_xlabel("TIC 0")
#ax.set_ylabel("TIC 1")
#ax.set_title("MD")
#ax.set_title("Timewarp")
#plot_tic01_background(ax, tics_model, tics)
ax.scatter(tics_model_short[0, 0], tics_model_short[0, 1], c="C1", alpha=0.5)

def update(i):
    #ax.clear()
    ax.scatter(tics_model_short[i+1, 0], tics_model_short[i+1, 1], c="C1", alpha=0.85)
    ax.plot(tics_model_short[i:i+2, 0], tics_model_short[i:i+2, 1], c="C1", alpha=0.85)
    #ax.set_xlim(-2, frames+2)
    #ax.set_ylim(min(y) - 1, max(y) + 1)

ani = animation.FuncAnimation(fig, update, frames=len(tics_model_short)-1, interval=50)
ani.save(base_dir+f"outputs/figures/Transitions-{protein}-model.gif", writer='pillow')
plt.show()
len(tics_model_short)


In [ ]:
np.where(tics_model_short[:, 0]<-3)

In [ ]:

import matplotlib.animation as animation
import numpy as np
import itertools 



fig, ax = plt.subplots(figsize=(4,4))
ax.set_ylim(tics[:,1].min(),tics[:,1].max())
ax.set_xlim(tics[:,0].min(),tics[:,0].max())
ax.set_xticks([])
ax.set_yticks([])
#ax.set_xlabel("TIC 0")
#ax.set_ylabel("TIC 1")
#ax.set_title("MD")
#ax.set_title("MD")
#plot_tic01_background(ax, tics, tics)
ax.scatter(tics_short[0, 0], tics_short[0, 1], alpha=0.5, c="C0")

def update(i):
    #ax.clear()
    ax.scatter(tics_short[i+1, 0], tics_short[i+1, 1], alpha=0.85, c="C0")
    ax.plot(tics_short[i:i+2, 0], tics_short[i:i+2, 1], alpha=0.85, c="C0")
    #ax.set_xlim(-2, frames+2)
    #ax.set_ylim(min(y) - 1, max(y) + 1)

ani = animation.FuncAnimation(fig, update, frames=len(tics_short)-1, interval=100)
ani.save(base_dir+f"outputs/figures/Transitions-{protein}-MD.gif", writer='pillow')

plt.show()
len(tics_short)


In [ ]:
trajectory_exploration = model_npz["positions"]
traj_model_exp = md.Trajectory(
    xyz=trajectory_exploration,
    topology=topology
)

In [ ]:

#indices = np.where(labels_md==4)[0]
indices = np.arange(len(tics_model_exp))[sampling_time_model_exp<max_time][2::10][:-53][::2]
select_model = traj_model_exp.slice(indices)

import nglview as nv
w = nv.show_mdtraj(select_model.superpose(trajectory,2000))
#w = nv.show_mdtraj(select)

w.clear_representations()
w.add_ball_and_stick()
#w.add_cartoon()
w

In [ ]:

#indices = np.where(labels_md==4)[0]
indices = np.arange(len(tics_model))[sampling_time_model<max_time][2::19][:-30][::2]
select_model = traj_model.slice(indices)

import nglview as nv
w = nv.show_mdtraj(select_model.superpose(trajectory,2000))
#w = nv.show_mdtraj(select)

w.clear_representations()
w.add_ball_and_stick()
w.add_cartoon()
w

In [ ]:
#indices = np.where(labels_md==4)[0]
indices = np.arange(len(tics))[sampling_time_md<max_time][::md_sapcing]
select = trajectory.slice(indices)

import nglview as nv
w = nv.show_mdtraj(select.superpose(trajectory,2000))
#w = nv.show_mdtraj(select)

w.clear_representations()
w.add_ball_and_stick()
w.add_cartoon()
w

In [ ]:
from time import sleep

for frame in range(0, len(indices)):
    # set frame to update coordinates
    w.frame = frame
    # make sure to let NGL spending enough time to update coordinates
    sleep(0.5)
    w.download_image(filename=f'model_{protein}-image-{frame:03d}.png')
    # make sure to let NGL spending enough time to render before going to next frame
    sleep(1.0)